https://medium.com/mlearning-ai/create-a-neural-network-with-pytorch-lightning-in-just-100-lines-of-code-43eccbf3fba

## **NN_CONTROLLER**



In [1]:
# PACKAGES

# data handling
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# saving models
import torch.onnx
import onnx


# logging metrics output
# from torch.utils.tensorboard import SummaryWriter
# import tensorboard

# else:
import copy
import matplotlib.pyplot as plt
import pandas as pd
from joblib import dump

In [2]:
class GetDataset(Dataset):
    "Geting features and outputs ndarray from file."
    def __init__(self, filename, ninputs, noutputs, delimiter: str=','):
        # data loading
        xy = np.loadtxt(filename, delimiter=delimiter, dtype=np.float32)
        self.X = xy[:, :ninputs]
        self.y = xy[:, ninputs:ninputs+noutputs]
        self.n_samples = xy.shape[0]
    
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    
class DatasetFromXy(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        self.n_samples = X.shape[0]

    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [135]:
# DATASET GENERATION
# filename = '../Datasets/dataset_5D_002.csv'
filename = '../Datasets/dataset_009.csv'
ninputs = 2
noutputs = 10

data_raw = GetDataset(filename=filename, ninputs=ninputs, noutputs=noutputs)
X_raw = data_raw.X
y_raw = data_raw.y

scalerX = StandardScaler()
X_scaled = scalerX.fit_transform(X_raw)

data_scaled = DatasetFromXy(X_scaled, y_raw)

test_set, train_set = random_split(data_scaled, lengths=[.2,.8], generator=torch.Generator())
nsamples = len(test_set)

print(f'Number of training samples: {len(train_set):4}\nNumber of testing samples: {len(test_set):5}')


Number of training samples:    7
Number of testing samples:     2


In [136]:
# DATALOADERS AND TRAIN/TEST BATCHES
train_batch_size = 4  # 400 para dataset_005, ~30 para dataset_006
test_batch_size = 10    # nos da igual

train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=False, num_workers=0)

## Network

In [5]:
# NEURAL NETWORK DEFINITION
class Network(pl.LightningModule):
    def __init__(self, input_size, output_size, hidden_layers, learning_rate, drop_p=0.2, leaky_slope=0.0):
        ''' Builds a fully connected network with arbitrary hidden layers.
        
            Arguments
            ---------
            input_size: integer, size of the input layer.
            output_size: integer, size of the output layer.
            hidden_layers: list of integers, the sizes of the hidden layers.
            learning_rate: learning rate for the optimizer.
            drop_p: float = 0.2, drop out probability.
            leacky_slope: float = 0.0 slope of the leakyReLU activation function. A value of zero is
            equivalent to using the ReLU function.
        '''
        super().__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_layers
        self.structure_dict = {'input_size': input_size,
                               'output_size': output_size,
                               'hidden_sizes': hidden_layers}

        # Input to a hidden layer
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        # Ouput layer
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        # Dropout probabilty
        self.dropout = nn.Dropout(p=drop_p)

        # LeakyReLU activation function negative slope
        self.leaky_slope = leaky_slope

        # Loss function
        self.loss_fun = nn.MSELoss()    # Mean Squared Error Loss

        # Optimizer learning rate
        self.learning_rate = learning_rate

        # optimizer:
        self.optimizer = []

        # new: ########################
        self.last_test_loss = 0

        # extra: monitoring training loss
        # self.training_loss = []
        # self.test_batch_idx = []
        

    def forward(self, x):
        ''' Forward pass through the network, returns the output logits.'''
        for each in self.hidden_layers:
            # x = F.relu(each(x))
            x = F.leaky_relu(each(x), self.leaky_slope)
            x = self.dropout(x)
        x = self.output(x)
        return x
    

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        self.optimizer = optimizer
        return optimizer
    

    def training_step(self, train_batch, batch_idx):
        "Definition of the training loop."
        X, y = train_batch                  # (extracting features and outputs)
        # y = y.type(torch.float32)         # (just in case)
        # forward pass
        y_pred = self.forward(X).squeeze()  # 
        # compute loss
        loss = self.loss_fun(y_pred, y)     # 
        self.log_dict({'train_loss': loss}, on_step=False, on_epoch=True, prog_bar=True,
                      logger=True, enable_graph=True)
        
        # extra: monitoring training loss:
        # self.training_loss.append(np.mean(loss.item()))
        # self.test_batch_idx.append(batch_idx)
        # extra:
        # writer.add_scalar('Train Loss', loss)
        return loss
    

    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        # forward pass
        y_pred = self.forward(X).squeeze()        
        # compute metrics
        loss = self.loss_fun(y_pred, y)
        rmse = np.sqrt(mean_squared_error(y_pred, y))
        r2 = r2_score(y_pred, y)

        self.last_test_loss = loss

        self.log_dict({'Cost function': loss},
                      on_step=False, on_epoch=True, prog_bar=True, logger=True)

        # self.log_dict({'Cost function': loss, 'r2': r2, 'Root Mean Square Error': rmse},
                    #   on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

## **RUN: ALL DEF+TRAIN+TEST**

In [137]:
# NETWORK DEFINITION
# Number of layers and their sizes:
input_size = ninputs
output_size = noutputs
hidden_layers = [10]
hidden_layers = [20]
hidden_layers = [20, 20]
hidden_layers = [20, 40, 20]
hidden_layers = [20, 40, 40, 20]
# Other hyperparameters:
max_epochs = 2000
lr = 0.005
drop_p = 0.
leaky_slope = 0.0

model = Network(input_size=input_size, output_size=output_size, hidden_layers=hidden_layers,
                learning_rate=lr, drop_p=drop_p, leaky_slope=leaky_slope)

patience = 20
early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=0.00, patience=patience, verbose=True, mode="min")
# DEFINITION OF THE TRAINER
trainer = pl.Trainer(accelerator='cpu', devices='auto', max_epochs=max_epochs,
                     callbacks=[early_stop_callback], log_every_n_steps=8, deterministic=True)

# TRAINING
trainer.fit(model=model, train_dataloaders=train_loader)
# TESTING
trainer.test(model=model, dataloaders=test_loader)



GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\ignac\anaconda3\lib\site-packages\pytorch_lightning\trainer\setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name          | Type       | Params
---------------------------------------------
0 | hidden_layers | ModuleList | 3.4 K 
1 | output        | Linear     | 210   
2 | dropout       | Dropout    | 0     
3 | loss_fun      | MSELoss    | 0     
---------------------------------------------
3.6 K     Trainable params
0         Non-trainable params
3.6 K     Total params
0.014     Total estimated model params size (MB)
c:\Users\ignac\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may 

Training: 0it [00:00, ?it/s]

Metric train_loss improved. New best score: 1.278
Metric train_loss improved by 0.070 >= min_delta = 0.0. New best score: 1.208
Metric train_loss improved by 0.072 >= min_delta = 0.0. New best score: 1.136
Metric train_loss improved by 0.096 >= min_delta = 0.0. New best score: 1.039
Metric train_loss improved by 0.129 >= min_delta = 0.0. New best score: 0.910
Metric train_loss improved by 0.154 >= min_delta = 0.0. New best score: 0.756
Metric train_loss improved by 0.168 >= min_delta = 0.0. New best score: 0.589
Metric train_loss improved by 0.132 >= min_delta = 0.0. New best score: 0.457
Metric train_loss improved by 0.073 >= min_delta = 0.0. New best score: 0.384
Metric train_loss improved by 0.050 >= min_delta = 0.0. New best score: 0.334
Metric train_loss improved by 0.056 >= min_delta = 0.0. New best score: 0.277
Metric train_loss improved by 0.062 >= min_delta = 0.0. New best score: 0.216
Metric train_loss improved by 0.031 >= min_delta = 0.0. New best score: 0.185
Metric train_l

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Cost function         0.05456965044140816
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'Cost function': 0.05456965044140816}]

### Network definition

In [99]:
# NETWORK DEFINITION

# Number of layers and their sizes:
input_size = ninputs
output_size = noutputs
hidden_layers = [10]

# Other hyperparameters:
max_epochs = 1000
lr = 0.005
drop_p = 0.

leaky_slope = 0.0

model = Network(input_size=input_size, output_size=output_size, hidden_layers=hidden_layers,
                learning_rate=lr, drop_p=drop_p, leaky_slope=leaky_slope)

# Including early stoping: `patience` is the key parameter here.
patience = 20
early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=0.00, patience=patience, verbose=True, mode="min")

### Training

In [97]:
# DEFINITION OF THE TRAINER
trainer = pl.Trainer(accelerator='cpu', devices='auto', max_epochs=max_epochs,
                     callbacks=[early_stop_callback], log_every_n_steps=8, deterministic=True)

# TRAINING
trainer.fit(model=model, train_dataloaders=train_loader)
# writer.flush()
# writer.close()


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\ignac\anaconda3\lib\site-packages\pytorch_lightning\trainer\setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name          | Type       | Params
---------------------------------------------
0 | hidden_layers | ModuleList | 60    
1 | output        | Linear     | 210   
2 | dropout       | Dropout    | 0     
3 | loss_fun      | MSELoss    | 0     
---------------------------------------------
270       Trainable params
0         Non-trainable params
270       Total params
0.001     Total estimated model params size (MB)
c:\Users\ignac\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may 

Training: 0it [00:00, ?it/s]

Metric train_loss improved. New best score: 1.305
Metric train_loss improved by 0.182 >= min_delta = 0.0. New best score: 1.123
Metric train_loss improved by 0.152 >= min_delta = 0.0. New best score: 0.971
Metric train_loss improved by 0.131 >= min_delta = 0.0. New best score: 0.841
Metric train_loss improved by 0.111 >= min_delta = 0.0. New best score: 0.729
Metric train_loss improved by 0.099 >= min_delta = 0.0. New best score: 0.630
Metric train_loss improved by 0.091 >= min_delta = 0.0. New best score: 0.540
Metric train_loss improved by 0.075 >= min_delta = 0.0. New best score: 0.464
Metric train_loss improved by 0.070 >= min_delta = 0.0. New best score: 0.394
Metric train_loss improved by 0.064 >= min_delta = 0.0. New best score: 0.330
Metric train_loss improved by 0.050 >= min_delta = 0.0. New best score: 0.279
Metric train_loss improved by 0.048 >= min_delta = 0.0. New best score: 0.231
Metric train_loss improved by 0.037 >= min_delta = 0.0. New best score: 0.195
Metric train_l

### Testing

In [98]:
# TEST OUTPUT
print(trainer.test(model=model, dataloaders=test_loader))

c:\Users\ignac\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Cost function        0.0010673918295651674
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'Cost function': 0.0010673918295651674}]


### Saving & Loading Model to Disk

In [126]:
# ----- IDENTIFIER ------
identifier = '009_20_40_40_20'
# ------ -------- -------

# SAVING THE STATE DICT
state_dict_path = '../Models/' + 'state_dict_' + identifier
torch.save(model.state_dict(), state_dict_path)

# SAVING THE STRUCTURE DICT
structure_dict = model.structure_dict
dump(structure_dict, '../Models/' + 'structure_' + identifier, compress=True)

# SAVING THE STANDARD SCALER
dump(scalerX, '../Models/scalerX_' + identifier[0:3] + '.bin', compress=True)

['../Models/scalerX_009.bin']

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [170]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("\nOptimizer's state_dict:")
for var_name in model.optimizer.state_dict():
    print(var_name, "\t", model.optimizer.state_dict()[var_name])

Model's state_dict:
hidden_layers.0.weight 	 torch.Size([20, 2])
hidden_layers.0.bias 	 torch.Size([20])
hidden_layers.1.weight 	 torch.Size([40, 20])
hidden_layers.1.bias 	 torch.Size([40])
hidden_layers.2.weight 	 torch.Size([40, 40])
hidden_layers.2.bias 	 torch.Size([40])
hidden_layers.3.weight 	 torch.Size([20, 40])
hidden_layers.3.bias 	 torch.Size([20])
output.weight 	 torch.Size([10, 20])
output.bias 	 torch.Size([10])

Optimizer's state_dict:
state 	 {0: {'step': tensor(21.), 'exp_avg': tensor([[-4.1757e-03, -4.4490e-03],
        [ 1.9473e-03, -1.2747e-03],
        [-7.8532e-03, -6.6654e-03],
        [-4.8333e-04,  2.4932e-03],
        [ 5.7345e-03, -7.8494e-04],
        [ 7.4421e-04,  6.1482e-03],
        [ 7.1190e-03, -6.3492e-03],
        [ 2.3397e-03,  1.6157e-03],
        [-2.6480e-03,  2.2710e-03],
        [ 8.2957e-04, -1.2661e-03],
        [ 2.7800e-03,  3.3480e-03],
        [-3.9422e-03, -2.4990e-03],
        [-7.4738e-04, -8.9402e-04],
        [-6.4684e-05,  7.9290e-

#############################################################################

From here, going to `QuickNN.ipynb` to do the inference of a single controller.

#############################################################################

###### Saving the model to disk (old)

In [ ]:
# x = torch.randn(train_batch_size, 1,requires_grad=True)
import sys
sys.exit("Reached kind of end.")

in_model = torch.tensor([10., 10])

model_name = '../Models/Model_002_list.onnx'


torch.onnx.export(model,                     # model being run
                  in_model,                  # model input (or a tuple for multiple inputs)
                  model_name,                # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input'  : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})


###### Loading Model from disk (old)

In [ ]:
# load_model_name = '../Models/Model_002_1.onnx'
# model2 = onnx.load(load_model_name)

### Particular Examples: Inference

In [43]:
# def PredictController(trim, model, scalerX=scalerX, scalery=scalery):
def PredictController(trim, model, scalerX=scalerX):
    """
    Function that receives the trim state vector and generates a prediction of the appropriate
    controller based on the trained model 'model'.

    Arguments
    ---------
    trim: State in the flight envelope
    model: Network used
    scalerX: scaler used for the features list from the dataset
    scalery: scaler used for the outputs list from the dataset
    """
    trim = np.array(trim).reshape(1,-1)
    features = scalerX.transform(trim)
    out_pred = model(torch.Tensor(features))    # torch.Tensor([1,10])
    # K_comps = scalery.inverse_transform(out_pred.detach().numpy())
    K_comps = out_pred.detach().numpy()
    state_dim = K_comps.size    # 10
    return K_comps.reshape(2, int(state_dim/2))


def ExportSingleController(filename='ExportedController.csv', K=0):
    np.savetxt(filename, K, delimiter=",")

### Predicting `Single Controller` based on single flight conditions:

In [ ]:
trim_conditions = [18.99, 1492]
K = PredictController(trim_conditions, model)
print(K)

[[ 9.6461666e-01 -1.7474092e+00  1.5793829e+00 -3.8410774e-03
   2.5257075e-01]
 [-7.2058612e-03  1.7754954e+00 -2.9745905e+01 -7.7651601e+00
  -9.2721909e-01]]


#### Exporting that single controller

In [ ]:
path = '../Controllers/'
basename = 'SingleController_002_test2_checkgood'
stringV = str(trim_conditions[0])
stringH = str(trim_conditions[1])
fullname = path + basename + '-' + stringV + '-' + stringH + '.csv'

ExportSingleController(filename=fullname, K=K)

# TO PASTE TO MATLAB:
print('trim_conditions = ' + str(trim_conditions) + ';')
print("controllername = '" + basename + "-" + stringV + "-" + stringH + ".csv';")


trim_conditions = [18.99, 1492];
controllername = 'SingleController_002_test2_checkgood-18.99-1492.csv';


### Generating the `Predicted Controllers` from the testing data to exporting them

In [ ]:
# GETTING A PREDICTED CONTROLLER FOR EVERY CONDITION IN THE TEST DATA
OUTMATRIX =  np.empty((nsamples, 10))
# reach = 0
for idx, (features, outputs) in enumerate(test_loader):
    # Controller prediction:
    out_pred = model(features)
    # print(out_pred.shape)
    np.append(OUTMATRIX, out_pred.detach().numpy(), axis=0)
    print(out_pred.detach().numpy().shape)
# # K_comps = scalery.inverse_transform(out_pred.detach().numpy())
# K_comps = out_pred.detach().numpy()
# # # Conditions
# conditions = scalerX.inverse_transform(features)
# add = np.concatenate((conditions, K_comps), axis=1)
# # OUTMATRIX[reach:reach+len(features),0:12] = np.concatenate((conditions, K_comps), axis=1)
# # reach += len(features)
# np.append(OUTMATRIX, add, axis=0)
# # if idx == 1: print(add.shape)
# # trim = np.array(trim).reshape(1,-1)
# # features = scalerX.transform(trim)
# # out_pred = model(torch.Tensor(features))    # torch.Tensor([1,10])
# # # K_comps = scalery.inverse_transform(out_pred.detach().numpy())
# # K_comps = out_pred.detach().numpy()
# # state_dim = K_comps.size    # 10
# # return K_comps.reshape(2, int(state_dim/2))
# # if idx == 0 : print(K_comps[0]); print(scalerX.inverse_transform(features)[0])
# # print(conditions)
# # if idx == 0 : print(scalerX.inverse_transform(features))
OUTMATRIX

(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(20, 10)
(15, 10)


array([[8.57291739e-312, 8.58140252e-312, 1.23714038e-319, ...,
        3.02907761e-152, 2.31462645e-152, 1.80723903e+185],
       [4.44732411e+252, 8.06742686e+276, 5.56319841e+180, ...,
        2.86745443e+161, 2.59459186e+161, 6.21452776e+175],
       [1.11493090e+277, 6.22651684e+228, 6.50949581e+252, ...,
        1.23039479e+224, 6.29199815e+233, 4.63456040e+228],
       ...,
       [1.96631590e-153, 1.51437960e+256, 6.19640460e+223, ...,
        3.28083902e+199, 1.40653076e+142, 8.37981733e+276],
       [3.03426193e-086, 1.33716990e-152, 1.28625698e+248, ...,
        6.36697561e+151, 4.81127985e+151, 2.41799184e+198],
       [8.37981727e+276, 6.15310389e+223, 1.78711426e+161, ...,
        2.12690333e-259, 9.70378172e+189, 2.46636231e-154]])

In [ ]:
predicted_controllers_name = 'PredictedControllers_002_test1.csv';
path = '../Controllers/'
np.savetxt(path + predicted_controllers_name, OUTMATRIX, delimiter=',')